## 1. PyTorch 数据管道全景：Dataset → Sampler → DataLoader → Batch

DataLoader 不是“读取数据”这么简单，它其实是一个“批处理调度器”。流程是：

1. Dataset：定义“第 i 个样本是什么”。

2. Sampler / BatchSampler：定义“按什么顺序取 i”（以及怎么把 i 组成 batch）。

3. DataLoader：并行取样本 + collate_fn 组 batch +（可选）pin memory +（你在训练 loop 里）搬到 GPU。

**Dataset 负责“样本内容”，Sampler 负责“索引策略”，collate_fn 负责“怎么拼 batch”.**

## 2. Dataset 两大类：Map-style vs Iterable-style

### 2.1 Map-style Dataset（最常见

满足：

__len__()：数据集大小

__getitem__(idx)：返回单个样本

典型：图片/文本分类、NER 样本、pair 数据等。

**关键理解：DataLoader 会不断产生 idx（来自 sampler），然后调用 dataset[idx] 拿样本**

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __Len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return {"x": x, "y": y}

### 2.2 IterableDataset（流式/无限/无法随机访问）

当你：

数据太大（不能 len / 不能随机访问）

数据来自流（Kafka、socket、巨型日志）

或者要做在线生成（无限数据）

用 IterableDataset，核心是实现 __iter__()：

**IterableDataset 通常 不能 shuffle（除非你自己做 buffer shuffle）。**

**多 worker 时，必须“切分数据流”，否则每个 worker 会读到同样的数据（重复样本）。后面会讲怎么避免。**

In [ ]:
import torch
from torch.utils.data import IterableDataset

class StreamDataset(IterableDataset):
    def __iter__(self):
        for i in range(10*12):
            yield torch.tensor([i]), torch.tensor(i % 2)

## DataLoader 的核心参数：不只是 batch_size + shuffle

### 3.1 基本参数

batch_size：每个 batch 样本数

shuffle=True：本质是用 RandomSampler

drop_last=True：丢掉最后不足 batch 的部分（分布式训练常用，保证步数一致）

collate_fn：把 list[样本] → batch（非常重要，NLP 变长序列必用）

### 3.2 采样相关（Sampler / BatchSampler）

sampler=：自定义索引顺序（如类别均衡、长度分桶）

batch_sampler=：直接产生“batch 的索引列表”，此时不能再传 batch_size/shuffle/sampler

>面试题：shuffle=True 和 sampler=... 能同时用吗？
>不能（逻辑冲突）。你要讲清：shuffle 其实就是一种 sampler

### 3.3 并行与性能

num_workers：并行加载数据的进程数（不是线程，默认多进程）

pin_memory=True：把 batch 放到“页锁定内存”，CPU→GPU 拷贝更快

persistent_workers=True：epoch 之间不销毁 worker，减少反复 fork 开销（训练大模型常用）

prefetch_factor：每个 worker 预取 batch 数（默认 2）

timeout：防止 worker 卡死（调试用）

**经验但可讲成原理：**

数据预处理重（tokenize、解压、增强）→ num_workers 开大；

数据很轻（纯 tensor index）→ num_workers=0 反而更稳；

pin_memory 对 GPU 训练通常有正收益，但注意内存占用上升。



## 4. collate_fn：NLP 的“命门”

默认 collate_fn 会尝试把同类型的样本 stack 成 tensor。

但 NLP 常见：每个样本长度不一样 → 你必须 padding + mask。

### 4.1 一个标准 NLP batch：input_ids + attention_mask + labels

假设 Dataset 返回：

In [ ]:
{"input_ids": [101, 2003, ...], "label": 1}

我们写 collate_fn：padding 到 batch 内最大长度。

In [ ]:
import torch

PAD_ID = 0

def collate_fn(batch):
    #batch: List[dict]
    input_ids = [torch.tensor(x["input_ids"], dtype=torch.long) for x in batch]
    labels = torch.tensor([x["label"] for x in batch], dtype=torch.long)

    lengths = torch.tensor([len(x) for x in input_ids], dtype=torch.long)
    max_len = int(lengths.max())

    padded = torch.full((len(input_ids), max_len), PAD_ID, dtype=torch.long)
    attention_mask = torch.zeros((len(input_ids), max_len), dtype=torch.long)

    for i, ids in enumerate(input_ids):
        #当前样本的真实长度（token数）
        L = ids.numel()

        #将真实token拷贝到padded tensor的第i行；只填充前L个位置，其余保持PAD
        padded[i, :L] = ids

        #真实token为1，padding位置为0
        attention_mask[i, :L] = 1

    return {
        "input_ids": padded,
        "attention_mask": attention_mask,
        "lengths": lengths,
        "labels": labels
    }

collate_fn 的输入是 list[样本]（每个样本是 __getitem__ 的返回）。

padding 发生在 CPU；之后训练 loop 再 .to(device, non_blocking=True)。

attention_mask/lengths 是后续模型/损失计算的关键。

### 4.2 动态 padding vs 固定 padding

动态 padding：每个 batch pad 到 batch max_len（更省算力，BERT/LLM 常用）

固定 padding：pad 到全局 max_len（实现简单，但浪费）

> 动态 padding 为什么会让 GPU 利用率不稳定？

>你要答：不同 batch 的 max_len 波动导致计算量波动；解决：按长度分桶 bucketing

## 5. 长度分桶（Bucketing）：让 batch 更整齐、更快

思路：把相近长度的样本放在同一 batch，减少 padding 浪费。

实现方式有两类：

1. 先排序/分桶，再 batch（batch_sampler）

2. Sampler 产生 idx 时就按桶策略来

**目标：** 降低 padding，提高吞吐

**代价：** 随机性降低（通常桶内 shuffle / 桶间 shuffle 折中）

## 6. 随机性与可复现：多 worker 的 seed 怎么搞

### 6.1 为什么 num_workers>0 会让随机增强不稳定？

因为每个 worker 是独立进程，有自己 RNG 状态。
如果不设置，会出现：

每次跑随机增强不一样（正常）

但更糟：不同 worker 可能拿到相同 seed → 增强重复（看实现）

### 6.2 标准做法：worker_init_fn + generator

In [ ]:
import torch
import random
import numpy as np

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    worker_init_fn=seed_worker,
    generator=g,   
)

7. 分布式训练（DDP）下 DataLoader 的关键：DistributedSampler

在多卡 DDP 下，每个进程（rank）必须拿到不同切片的数据，否则会重复训练同一数据。

In [ ]:
from torch.utils.data.distributed import DistributedSampler

sampler = DistributedSampler(dataset, shuffle=True)

loader = DataLoader(dataset, batch_size=32, sampler=sampler, num_workers=4)

for epoch in range(E):
    sampler.set_epoch(epoch)    #关键：每一个epoch重新洗牌一致
    for batch in loader:
        ...

> 为什么必须 set_epoch

> 否则每个 epoch shuffle 的顺序不变（或者各 rank 不一致），影响训练效果/可复现。

>drop_last 为什么常配合 DDP？

>保证每个 rank step 数一致，否则最后一个 batch 大小不一可能导致同步/shape 问题。

## 8. IterableDataset + 多 worker：避免重复读

如果是 IterableDataset，多 worker 时，每个 worker 都会跑一遍 __iter__，导致重复数据。

常见做法：用 get_worker_info() 切分流：

In [ ]:
from torch.utils.data import IterableDataset, get_worker_info

class ShardedStream(IterableDataset):
    def __init__(self, data):
        self.data = data

    def __iter__(self):
        info = get_worker_info()
        if info is None:
            #单进程
            for x in self.data:
                yield x
        else:
            #多worker：按worker_id stride切分
            wid = info.id
            nw = info.num_workers
            for i in range(wid, len(self.data), nw):
                yield self.data[i]

IndentationError: expected an indented block (2933166974.py, line 9)

## 9. 性能调优：你要能像工程师一样解释瓶颈在哪里

### 9.1 常见瓶颈

训练 loop 卡在取 batch：CPU 处理/IO 慢

GPU 利用率低：数据供不上（dataloader 慢）

多 worker 反而更慢：进程间拷贝/序列化开销大、磁盘随机读更糟

### 9.2 你应该会的调参逻辑

先 num_workers=0 跑通（排除多进程坑）

再逐步加：2 → 4 → 8，观察吞吐

GPU 训练常开：pin_memory=True，搬运用 non_blocking=True

epoch 间卡顿：试 persistent_workers=True

典型训练 loop：

In [ ]:
for batch in loader:
    batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
    ...

任务 A（基础）：写一个文本分类 Dataset：

输入：texts: List[str]，labels: List[int]

__getitem__ 返回 {"input_ids": List[int], "label": int}（你可以用简化 tokenizer：按空格分词再映射 id）

任务 B（核心）：写 collate_fn：动态 padding + attention_mask。

输出 shape：input_ids [B, T]，attention_mask [B, T]，labels [B]

任务 C（进阶）：

做长度分桶 batch（实现一个简单 bucket batch sampler 或者排序后分 batch）。

In [ ]:
from typing import List, Dict, Any, Iterator, Optional
import math
import random
import torch
from torch.utils.data import Dataset, DataLoader, Sampler

class SimpleVocab: #一个极简词表：token(str)->id(int)
    def __init__(self, texts: List[str], pad_token: str="<PAD>", unk_token: str="<UNK>"):
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.token_to_id: Dict[str, int] = {}   #token到id的映射表
        self.id_to_token: List[str] = []    #id到token的反查表
        self._add_token(self.pad_token) #保证PAD id=0
        self._add_token(self.unk_token) #保证UNK id=0
        for text in texts:  #遍历所有文本构建词表
            for tok in self._basic_tokenize(text):  #逐token加入词表
                self._add_token(tok)    #加入词表，若已存在则忽略
    
    def _basic_tokenize(self, text: str) -> List[str]: #简化tokenizer，按空格切分
        text = text.strip() #去掉首位空格
        if text == "":  #如果整句为空
            return []   #返回空的token列表
        return text.strip() #按任意空白切分
    
    def _add_token_(self, token: str) -> None:  #把token加入词表
        if token in self.token_to_id:
            return
        new_id = len(self.id_to_token)  #新token的id等于当前词表的大小
        self.token_to_id[token] = new_id
        self.id_to_token.append(token)

    @property
    def pad_id(self) -> int:    #暴露pad id，collate_fn会用
        return self.token_to_id[self.pad_token]
    
    @property
    def unk_id(self) -> int:    #暴露unk id，遇到OOV时会用
        return self.token_to_id[self.unk_token]
    
    def encode(self, text: str) -> List[int]:   #把文本编成 token id 序列
        tokens = self._basic_tokenize(text) #分词
        ids: List[int] = [] #初始化id列表
        for tok in tokens:
            ids.append(self.token_to_id.get(tok, self.unk_id))
        return ids  #反汇变长的序列
    
class TextDataset(Dataset): #任务A：文本分类数据集
    def __init__(self, texts: List[str], labels: List[int], vocab: Optional[SimpleVocab] = None) -> None:
        assert len(texts) == len(labels)    #防止索引错位
        self.texts = texts
        self.labels = labels
        self.vocab = vocab if vocab is not None else SimpleVocab    #若没传vocab，则用全量texts构建词表
        self.lengths: List[int] = []    #任务C会用到，每条样本的序列长度
        for t in self.texts:
            self.lengths.append(len(self.vocab.encode(t)))  #预计算编码后的长度（bucketing需要）

    def __len__(self) -> int:   #Dataset必备，返回样本长度
        return len(self.texts)  #返回数据集大小
    
    def __getitem__(self, idx: int) -> Dict[str, any]:  #Dataset必备，返回第idx个样本
        text = self.texts[idx]
        label = int(self.labels[idx])
        input_ids = self.vocab.encode(text) #文本编码成 List[int]
        return {"input_ids": input_ids, "label": label}
        
def make_collate_fn(pad_id: int): #任务B，返回一个闭包collate_fn(把pad_id带进去)
    def collate_fn(batch: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_ids_list = [torch.tensor(x["input_ids"], dtype=torch.long) for x in batch]    #把每条序列转换为tensor
        labels = torch.tensor([x["label"] for x in batch], dtype=torch.long)    #labels直接堆成[B]
        lengths = torch.tensor([t.numel() for t in input_ids_list], dtype=torch.long)   #每条序列的真实长度[B]
        max_len = int(lengths.max.item()) if len(batch) > 0 else 0  #batch内的最大长度，动态padding关键
        B = len(batch)  #batch size
        padded = torch.full((B, max_len), pad_id, dtype=torch.long) #先用PAD填满[B, T]
        attention_mask = torch.zeros((B, max_len), dtype=torch.long)    #mask初始全0， [B, T]
        for i , ids in enumerate(input_ids_list):
            L = ids.numel()
            if L == 0:
                continue    #处理空句子，保持PAD, mask全是0
            padded[i, :L] = ids #把真实token拷贝到前L位
            attention_mask[i, :L] = 1   #把真实token标1

        return {"input_ids": padded, "attention_mask": attention_mask, "labels":labels} #输出模型可以吃的batch
    return collate_fn

class BucketBatchSampler(Sampler[List[int]]):   #任务C，长度分桶batch sampler（返回“索引列表”）
    def __init__(
            self,
            lengths: List[int], #每个batch的长度
            batch_size: int,    #每个batch的样本数
            shuffle: bool = True,    #是否做随机
            drop_last: bool = False,    #丢弃最后不满batch的部分
            bucket_size_multiplier: int = 50,   #"局部窗口"大小 = batch size * multiplier(越大越随机，越小越整齐)
            seed: int = 42,
    ) -> None:
        self.lengths = lengths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.droplast = drop_last
        self.bucket_size = max(batch_size * bucket_size_multiplier, batch_size) #每个同窗口的样本数
        self.seed = seed
        self.epoch = 0  #epoch计数

    def set_epoch(self, epoch: int) -> None:
        self.epoch = epoch

    def __iter__(self) -> Iterator[List[int]]:  #迭代器：每次 yield 一个 batch 的 indices
        n = len(self.lengths)   #样本总数
        indices = list(range(n))    #构造所有样本的索引
        indices.sort(key=lambda i: self.lengths[i]) #先按全局长度排序（实现“相近长度靠近”）
        rng = random.Random(self.seed + self.epoch) #用 seed + epoch 构造可复现 rng
        if self.shuffle:
            buckets: List[List[int]] = []   #保存按窗口切分后的桶
            for start in range(0, n, self.bucket_size): #每次取一段窗口
                end = min(start + self.bucket_size, n)  #计算窗口右边界
                bucket = indices[start: end]    #切出一个窗口的长度
                rng.shuffle(bucket) #窗口内打乱
                buckets.append(bucket)  #收集窗口
            indices = [i for bucket in buckets for i in bucket] #把所有窗口拼回一个索引序列
            batch_starts = list(range(0, n, self.batch_size))   #每个batch的起始位置
            rng.shuffle(batch_starts)   #打乱batch的顺序（避免短的都在前面）
            for s in batch_starts:  #按照打乱后的batch起点顺序产出batch
                e = min(s + self.batch_size, n) #计算batch的终点
                batch = indices[s: e]   #切出一个batch
                if self.drop_last and len(batch) < self.batch_size: #如果不满batch，且drop_last
                    continue    #跳过最后不满batch的部分
                yield batch #产出一个batch的indices
        else:   #如果不需要随机性，完全按照长度顺序分batch
            for start in range(0, n ,self.batch_size):  #从头到尾按batch_size切
                end = min(start + self.batch_size, n)
                batch = indices[start:end]
                if self.droplast and len(batch) < self.batch_size:
                    continue
                yield batch

    def __len__(self) -> int:   #返回一个epoch有多少个batch（让DataLoader/进度条知道长度）
        n = len(self.lengths)
        if self.drop_last:
            return n // self.batch_size #向下取整
        return math.ceil(n / self.batch_size)   #否则向上取整
    
if __name__ == "__main__":
    texts = [  # 构造一些示例文本
    "i love nlp",  # 短句
    "pytorch dataloader is important",  # 长句
    "nlp interview",  # 中等
    "",  # 空句（测试边界条件）
    "bucketing reduces padding waste",  # 较长
    "hello world",  # 短
    ]
    labels = [1, 0, 1, 0, 0, 1]  # 构造示例标签（0/1 二分类